In [2]:
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, ParameterSampler

import joblib

try:
    import lightgbm as lgb
    HAS_LGB = True
except Exception:
    HAS_LGB = False


In [3]:

LABEL = "fraud"

TRAIN_PATH = "../../DATA/dataset/TRAIN_stage1"
TEST_PATH  = "../../DATA/dataset/TEST_stage1"

OUT_DIR = "artifacts/stage1_models"
OUT_DIR_METRICS = "artifacts/stage1_metrics"


In [4]:

from pathlib import Path

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
Path(OUT_DIR_METRICS).mkdir(parents=True, exist_ok=True)

In [5]:

def load_stage_df(path: str, label: str = LABEL):
    df = pd.read_parquet(path)
    if label not in df.columns:
        raise KeyError(f"Missing label column: {label}")
    X = df.drop(columns=[label])
    y = df[label].astype(np.int8).to_numpy()
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    return X, y


In [6]:
def topk_metrics(y_true, score, top_pct_list=(0.001, 0.002, 0.005, 0.01)):
    y = np.asarray(y_true).astype(int)
    s = np.asarray(score).astype(float)
    n = len(y)
    base_rate = float(y.mean()) if n else np.nan
    order = np.argsort(-s)
    y_sorted = y[order]

    rows = []
    for p in top_pct_list:
        k = max(int(np.ceil(n * p)), 1)
        top_y = y_sorted[:k]
        prec = float(top_y.mean())
        rec = float(top_y.sum() / max(y.sum(), 1))
        lift = float(prec / base_rate) if base_rate and base_rate > 0 else np.nan
        rows.append({"top_pct": p, "k": k, "precision": prec, "recall": rec, "lift": lift, "base_rate": base_rate})
    return pd.DataFrame(rows)


def evaluate_metrics(y_true, score):
    return {
        "auc": float(roc_auc_score(y_true, score)),
        "prauc": float(average_precision_score(y_true, score)),
        "base_rate": float(np.mean(y_true)),
    }


In [7]:

def fit_logit(X_tr, y_tr):
    model = Pipeline([
        ("scaler", StandardScaler(with_mean=False)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            max_iter=2000,
            class_weight="balanced",
        ))
    ])
    model.fit(X_tr, y_tr)
    return model


def fit_hgb(X_tr, y_tr):
    model = HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=6,
        max_iter=400,
        min_samples_leaf=200,
        l2_regularization=0.0,
        random_state=42,
    )
    model.fit(X_tr, y_tr)
    return model


def fit_lgb_small(X_tr, y_tr):
    if not HAS_LGB:
        raise RuntimeError("lightgbm is not available in this environment.")
    model = lgb.LGBMClassifier(
        objective="binary",
        n_estimators=600,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=6,
        min_data_in_leaf=300,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
    )
    model.fit(X_tr, y_tr)
    return model


def predict_score(model, X):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    if hasattr(model, "decision_function"):
        s = model.decision_function(X)
        return 1 / (1 + np.exp(-s))
    return model.predict(X)


In [20]:
X_tr, y_tr = load_stage_df(TRAIN_PATH)
X_te, y_te = load_stage_df(TEST_PATH)

print("train:", X_tr.shape, "test:", X_te.shape)
print("base_rate train:", float(y_tr.mean()), "test:", float(y_te.mean()))


train: (5332979, 26) test: (932762, 26)
base_rate train: 0.0014481587120444314 test: 0.002624463689558537


In [12]:

from sklearn.metrics import classification_report

candidates = [
    ("logit", fit_logit),
    ("hgb", fit_hgb),
]
if HAS_LGB:
    candidates.append(("lgb_small", fit_lgb_small))

results = []
topk_all = {}
reports = {}

for name, fit_fn in tqdm(candidates, desc="Training Stage1 models"):
    model = fit_fn(X_tr, y_tr)
    score_te = predict_score(model, X_te)

    m = evaluate_metrics(y_te, score_te)
    m["model"] = name
    results.append(m)

    topk = topk_metrics(y_te, score_te)
    topk_all[name] = topk
    topk.to_csv(Path(OUT_DIR_METRICS) / f"{name}_topk.csv", index=False)

    joblib.dump(model, Path(OUT_DIR) / f"{name}.joblib")
    np.save(Path(OUT_DIR_METRICS) / f"{name}_test_scores.npy", score_te)

    thr = np.quantile(score_te, 0.99)
    y_pred = (score_te >= thr).astype(int)

    rep_txt = classification_report(y_te, y_pred, digits=4)
    reports[name] = rep_txt

    print("\n" + "=" * 80)
    print(f"[{name}] threshold=quantile(0.99) -> top 1% as positive")
    print(rep_txt)

results_df = pd.DataFrame(results).sort_values(["prauc", "auc"], ascending=False).reset_index(drop=True)
results_df


Training Stage1 models:   0%|          | 0/3 [00:00<?, ?it/s]


[logit] threshold=quantile(0.99) -> top 1% as positive
              precision    recall  f1-score   support

           0     0.9990    0.9916    0.9953    930314
           1     0.1613    0.6148    0.2556      2448

    accuracy                         0.9906    932762
   macro avg     0.5802    0.8032    0.6254    932762
weighted avg     0.9968    0.9906    0.9933    932762


[hgb] threshold=quantile(0.99) -> top 1% as positive
              precision    recall  f1-score   support

           0     0.9988    0.9914    0.9951    930314
           1     0.1422    0.5445    0.2255      2448

    accuracy                         0.9902    932762
   macro avg     0.5705    0.7679    0.6103    932762
weighted avg     0.9965    0.9902    0.9930    932762

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_

,auc,prauc,base_rate,model
0,0.961857,0.465122,0.002624,lgb_small
1,0.857054,0.353443,0.002624,hgb
2,0.956060,0.304669,0.002624,logit


In [10]:

for name, _ in candidates:
    print(name)
    display(topk_all[name])


logit


,top_pct,k,precision,recall,lift,base_rate
0,0.001,933,0.508039,0.193627,193.578058,0.002624
1,0.002,1866,0.458735,0.349673,174.792002,0.002624
2,0.005,4664,0.285163,0.543301,108.655704,0.002624
3,0.010,9328,0.161342,0.614788,61.476254,0.002624


hgb


,top_pct,k,precision,recall,lift,base_rate
0,0.001,933,0.693462,0.264297,264.229966,0.002624
1,0.002,1866,0.577170,0.439951,219.919376,0.002624
2,0.005,4664,0.254074,0.484069,96.809782,0.002624
3,0.010,9328,0.142903,0.544526,54.450396,0.002624


lgb_small


,top_pct,k,precision,recall,lift,base_rate
0,0.001,933,0.807074,0.307598,307.519574,0.002624
1,0.002,1866,0.647374,0.493464,246.669087,0.002624
2,0.005,4664,0.293310,0.558824,111.760153,0.002624
3,0.010,9328,0.160806,0.612745,61.272014,0.002624


In [11]:

results_df.to_csv(Path(OUT_DIR_METRICS) / "stage1_summary.csv", index=False)
print("saved:", str(Path(OUT_DIR_METRICS) / "stage1_summary.csv"))


saved: artifacts/stage1_metrics/stage1_summary.csv


In [13]:
OUT_DIR = "artifacts/stage1_lgb_tuned"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

---

In [20]:

def load_stage_df(path: str, label: str = LABEL):
    df = pd.read_parquet(path)
    if label not in df.columns:
        raise KeyError(f"Missing label column: {label}")
    X = df.drop(columns=[label]).replace([np.inf, -np.inf], np.nan).fillna(0)
    y = df[label].astype(np.int8).to_numpy()
    return X, y

X_tr_full, y_tr_full = load_stage_df(TRAIN_PATH)
X_te, y_te = load_stage_df(TEST_PATH)

print("train:", X_tr_full.shape, "test:", X_te.shape)
print("base_rate train:", float(y_tr_full.mean()), "test:", float(y_te.mean()))


train: (5332979, 25) test: (932762, 25)
base_rate train: 0.0014481587120444314 test: 0.002624463689558537


In [25]:

X_tr, X_va, y_tr, y_va = train_test_split(
    X_tr_full, y_tr_full,
    test_size=0.2,
    random_state=42,
    stratify=y_tr_full
)

pos = float(np.sum(y_tr == 1))
neg = float(np.sum(y_tr == 0))
scale_pos_weight = (neg / max(pos, 1.0))

scale_pos_weight


689.576723858854

In [26]:

def fit_eval_lgb(params, X_tr, y_tr, X_va, y_va):
    clf = lgb.LGBMClassifier(
        objective="binary",
        n_estimators=5000,
        n_jobs=-1,
        random_state=42,
        scale_pos_weight=scale_pos_weight,
        **params
    )

    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="average_precision",
        callbacks=[
            lgb.early_stopping(stopping_rounds=200, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

    score_va = clf.predict_proba(X_va)[:, 1]
    prauc = float(average_precision_score(y_va, score_va))
    auc = float(roc_auc_score(y_va, score_va))
    best_iter = int(getattr(clf, "best_iteration_", clf.n_estimators))
    return clf, prauc, auc, best_iter


In [27]:

param_space = {
    "learning_rate": [0.02, 0.03, 0.05, 0.07],
    "num_leaves": [31, 63, 127, 255],
    "max_depth": [-1, 6, 7, 8, 10],
    "min_data_in_leaf": [50, 100, 200, 300, 500],
    "feature_fraction": [0.6, 0.7, 0.8, 0.9, 1.0],
    "bagging_fraction": [0.6, 0.7, 0.8, 0.9, 1.0],
    "bagging_freq": [0, 1],
    "reg_alpha": [0.0, 0.1, 0.5, 1.0],
    "reg_lambda": [0.0, 0.5, 1.0, 2.0, 5.0],
}

N_TRIALS = 30
sampler = list(ParameterSampler(param_space, n_iter=N_TRIALS, random_state=42))

best = {"prauc": -1.0, "auc": -1.0, "params": None, "best_iter": None, "model": None}
trial_rows = []

for i, params in tqdm(list(enumerate(sampler, start=1)), total=len(sampler), desc="Tuning LGBM (valid PR-AUC)"):
    model, prauc, auc, best_iter = fit_eval_lgb(params, X_tr, y_tr, X_va, y_va)
    trial_rows.append({
        "trial": i,
        "prauc_valid": prauc,
        "auc_valid": auc,
        "best_iter": best_iter,
        **params
    })

    if prauc > best["prauc"]:
        best.update({"prauc": prauc, "auc": auc, "params": params, "best_iter": best_iter, "model": model})

trials_df = pd.DataFrame(trial_rows).sort_values(["prauc_valid", "auc_valid"], ascending=False).reset_index(drop=True)
trials_df.head(10)


Tuning LGBM (valid PR-AUC):   0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Info] Number of positive: 6178, number of 

,trial,prauc_valid,auc_valid,best_iter,reg_lambda,reg_alpha,num_leaves,min_data_in_leaf,max_depth,learning_rate,feature_fraction,bagging_freq,bagging_fraction
0,21,0.768242,0.976399,2414,5.0,1.0,63,300,10,0.02,0.6,0,1.0
1,30,0.765424,0.981301,2198,5.0,1.0,127,500,6,0.02,0.9,0,0.8
2,10,0.764365,0.976049,2067,1.0,1.0,127,100,8,0.02,0.8,1,0.7
3,17,0.762402,0.974928,3430,0.0,0.5,127,300,7,0.02,0.6,1,0.6
4,1,0.762290,0.978662,2079,2.0,1.0,63,500,10,0.02,0.6,1,0.7
5,18,0.761763,0.978801,2336,0.0,0.1,31,200,8,0.02,0.7,0,1.0
6,3,0.646587,0.947062,3126,1.0,0.5,31,500,10,0.03,0.7,1,0.7
7,19,0.573052,0.967785,1,0.0,0.0,63,50,7,0.02,0.9,1,0.6
8,2,0.421928,0.974473,1,1.0,0.1,255,50,7,0.03,0.9,1,0.7
9,22,0.421885,0.976262,1,1.0,0.1,63,50,-1,0.03,1.0,0,0.8


In [28]:
best["prauc"], best["auc"], best["best_iter"], best["params"]

(0.7682423673122192,
 0.9763989647924208,
 2414,
 {'reg_lambda': 5.0,
  'reg_alpha': 1.0,
  'num_leaves': 63,
  'min_data_in_leaf': 300,
  'max_depth': 10,
  'learning_rate': 0.02,
  'feature_fraction': 0.6,
  'bagging_freq': 0,
  'bagging_fraction': 1.0})

In [29]:

best_model = best["model"]
score_te = best_model.predict_proba(X_te)[:, 1]

print("TEST AUC:", float(roc_auc_score(y_te, score_te)))
print("TEST PR-AUC:", float(average_precision_score(y_te, score_te)))


[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
TEST AUC: 0.9346556198588147
TEST PR-AUC: 0.5021281682942539


In [30]:

def report_at_top_pct(y_true, score, top_pct):
    thr = float(np.quantile(score, 1 - top_pct))
    y_pred = (score >= thr).astype(int)
    print(f"\nTop {top_pct*100:.3f}% (threshold={thr:.6f})")
    print(classification_report(y_true, y_pred, digits=4))

for p in [0.01, 0.005, 0.002, 0.001, 0.0005]:
    report_at_top_pct(y_te, score_te, p)



Top 1.000% (threshold=0.494931)
              precision    recall  f1-score   support

           0     0.9989    0.9916    0.9952    930314
           1     0.1573    0.5993    0.2492      2448

    accuracy                         0.9905    932762
   macro avg     0.5781    0.7954    0.6222    932762
weighted avg     0.9967    0.9905    0.9933    932762


Top 0.500% (threshold=0.763214)
              precision    recall  f1-score   support

           0     0.9988    0.9965    0.9976    930314
           1     0.2920    0.5564    0.3830      2448

    accuracy                         0.9953    932762
   macro avg     0.6454    0.7764    0.6903    932762
weighted avg     0.9970    0.9953    0.9960    932762


Top 0.200% (threshold=0.995016)
              precision    recall  f1-score   support

           0     0.9987    0.9993    0.9990    930314
           1     0.6501    0.4955    0.5624      2448

    accuracy                         0.9980    932762
   macro avg     0.8244    0.

In [31]:

joblib.dump(best_model, Path(OUT_DIR) / "lgb_stage1_tuned.joblib")
np.save(Path(OUT_DIR) / "lgb_stage1_tuned_test_scores.npy", score_te)
trials_df.to_csv(Path(OUT_DIR) / "lgb_tuning_trials.csv", index=False)

with open(Path(OUT_DIR) / "lgb_best_params.json", "w", encoding="utf-8") as f:
    import json
    json.dump(
        {"best_valid_prauc": best["prauc"], "best_valid_auc": best["auc"], "best_iter": best["best_iter"], "params": best["params"]},
        f, ensure_ascii=False, indent=2
    )

print("saved:", OUT_DIR)


saved: artifacts/stage1_lgb_tuned


### Stage1의 역할
> “확실히 아닌 애들은 통과시키고,
> 위험해 보이는 애들만 소수로 남긴다”

-> recall이 1순위 아님 (recall은 stage2가 보완)\
=> Precision이 더 중요하게 작용


In [34]:
MODEL_PATH = Path(OUT_DIR) / "lgb_stage1_tuned.joblib"

model = joblib.load(MODEL_PATH)
print("model loaded:", MODEL_PATH)


model loaded: artifacts/stage1_lgb_tuned/lgb_stage1_tuned.joblib


In [35]:
score_te = model.predict_proba(X_te)[:, 1]

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


In [38]:
sweep = sweep_threshold_by_top_pct(y_te, score_te)
best_row, _ = pick_best_under_recall(sweep, min_recall=0.50)

best_row


,top_pct,threshold,precision,recall,tp,fp,fn,alerts
12,0.002193,0.990359,0.607038,0.507353,1242,804,1206,2046


In [40]:
if hasattr(best_row, "iloc"):
    row = best_row.iloc[0]
else:
    row = best_row

thr = float(row["threshold"])

y_pred = (np.asarray(score_te) >= thr).astype(int)

print(f"Selected threshold = {thr:.6f}")
print(f"top_pct = {float(row['top_pct']):.6f} | precision = {float(row['precision']):.4f} | recall = {float(row['recall']):.4f}\n")
print(classification_report(y_te, y_pred, digits=4))


Selected threshold = 0.990359
top_pct = 0.002193 | precision = 0.6070 | recall = 0.5074

              precision    recall  f1-score   support

           0     0.9987    0.9991    0.9989    930314
           1     0.6070    0.5074    0.5527      2448

    accuracy                         0.9978    932762
   macro avg     0.8029    0.7532    0.7758    932762
weighted avg     0.9977    0.9978    0.9977    932762



### Compare with baseline

In [43]:
LABEL = "fraud"

BASE_TRAIN_PATH = "../../DATA/dataset/train_stage1"
BASE_TEST_PATH  = "../../DATA/dataset/test_stage1"
OUT_DIR = "artifacts/stage1_baseline_lgb"

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)


In [45]:
from sklearn.preprocessing import OneHotEncoder

def prep_baseline_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    drop_cols = [c for c in ["id", "date", "amount"] if c in df.columns]
    df.drop(columns=drop_cols, inplace=True)

    if "mcc" in df.columns:
        df["mcc"] = df["mcc"].astype("string").fillna("UNK")
    return df


def onehot_fit_transform(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols):
    enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    Xtr_cat = enc.fit_transform(train_df[cat_cols])
    Xte_cat = enc.transform(test_df[cat_cols])

    cat_names = enc.get_feature_names_out(cat_cols)
    Xtr_cat = pd.DataFrame(Xtr_cat, columns=cat_names, index=train_df.index)
    Xte_cat = pd.DataFrame(Xte_cat, columns=cat_names, index=test_df.index)

    Xtr_num = train_df.drop(columns=cat_cols)
    Xte_num = test_df.drop(columns=cat_cols)

    Xtr = pd.concat([Xtr_num, Xtr_cat], axis=1)
    Xte = pd.concat([Xte_num, Xte_cat], axis=1)

    return Xtr, Xte, enc


In [51]:
# Cell 2
LABEL = "fraud"

BASE_TRAIN_PATH = "../../DATA/dataset/train_stage1"
BASE_TEST_PATH  = "../../DATA/dataset/test_stage1"


In [52]:
# Cell 3
def prep_baseline_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    drop_cols = [c for c in ["id", "date", "amount"] if c in df.columns]
    df.drop(columns=drop_cols, inplace=True)

    if "mcc" in df.columns:
        df["mcc"] = df["mcc"].astype("string").fillna("UNK")
    return df


def onehot_fit_transform(train_df: pd.DataFrame, test_df: pd.DataFrame, cat_cols):
    enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    Xtr_cat = enc.fit_transform(train_df[cat_cols])
    Xte_cat = enc.transform(test_df[cat_cols])

    cat_names = enc.get_feature_names_out(cat_cols)
    Xtr_cat = pd.DataFrame(Xtr_cat, columns=cat_names, index=train_df.index)
    Xte_cat = pd.DataFrame(Xte_cat, columns=cat_names, index=test_df.index)

    Xtr_num = train_df.drop(columns=cat_cols)
    Xte_num = test_df.drop(columns=cat_cols)

    Xtr = pd.concat([Xtr_num, Xtr_cat], axis=1)
    Xte = pd.concat([Xte_num, Xte_cat], axis=1)

    return Xtr, Xte


In [53]:
# Cell 4
raw_train = pd.read_parquet(BASE_TRAIN_PATH)
raw_test  = pd.read_parquet(BASE_TEST_PATH)

train_df = prep_baseline_df(raw_train)
test_df  = prep_baseline_df(raw_test)

y_tr_full = train_df[LABEL].astype(np.int8).to_numpy()
y_te = test_df[LABEL].astype(np.int8).to_numpy()

X_tr_full = train_df.drop(columns=[LABEL])
X_te = test_df.drop(columns=[LABEL])

cat_cols = [c for c in ["mcc"] if c in X_tr_full.columns]
if len(cat_cols) > 0:
    X_tr_full, X_te = onehot_fit_transform(X_tr_full, X_te, cat_cols)

X_tr_full = X_tr_full.replace([np.inf, -np.inf], np.nan).fillna(0)
X_te      = X_te.replace([np.inf, -np.inf], np.nan).fillna(0)

print("train:", X_tr_full.shape, "test:", X_te.shape)
print("base_rate train:", float(y_tr_full.mean()), "test:", float(y_te.mean()))


train: (5332979, 127) test: (932762, 127)
base_rate train: 0.0014481587120444314 test: 0.002624463689558537


In [54]:
# Cell 5
X_tr, X_va, y_tr, y_va = train_test_split(
    X_tr_full, y_tr_full,
    test_size=0.2,
    random_state=42,
    stratify=y_tr_full
)

pos = float(np.sum(y_tr == 1))
neg = float(np.sum(y_tr == 0))
scale_pos_weight = (neg / max(pos, 1.0))
scale_pos_weight


689.576723858854

In [55]:

clf = lgb.LGBMClassifier(
    objective="binary",
    n_estimators=5000,
    learning_rate=0.05,
    num_leaves=64,
    min_data_in_leaf=300,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    reg_alpha=0.0,
    reg_lambda=0.0,
    n_jobs=-1,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
)

clf.fit(
    X_tr, y_tr,
    eval_set=[(X_va, y_va)],
    eval_metric="average_precision",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200, verbose=False),
        lgb.log_evaluation(period=0),
    ],
)

print("best_iter:", int(getattr(clf, "best_iteration_", clf.n_estimators)))


[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 6178, number of 

In [56]:

score_te = clf.predict_proba(X_te)[:, 1]

print("TEST AUC:", float(roc_auc_score(y_te, score_te)))
print("TEST PR-AUC:", float(average_precision_score(y_te, score_te)))


[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
TEST AUC: 0.8837094649475365
TEST PR-AUC: 0.05856981341846959


In [ ]:

def sweep_threshold_by_top_pct(y_true, score, top_pct_grid=None):
    if top_pct_grid is None:
        top_pct_grid = np.array(list(np.linspace(0.0002, 0.01, 80)))
    y = np.asarray(y_true).astype(int)
    s = np.asarray(score).astype(float)

    rows = []
    for top_pct in top_pct_grid:
        thr = float(np.quantile(s, 1 - top_pct))
        y_pred = (s >= thr).astype(int)

        tp = int(((y_pred == 1) & (y == 1)).sum())
        fp = int(((y_pred == 1) & (y == 0)).sum())
        fn = int(((y_pred == 0) & (y == 1)).sum())

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        rows.append({
            "top_pct": float(top_pct),
            "threshold": thr,
            "precision": float(precision),
            "recall": float(recall),
            "alerts": int(y_pred.sum()),
            "tp": tp, "fp": fp, "fn": fn
        })

    return pd.DataFrame(rows).sort_values("top_pct").reset_index(drop=True)


def pick_best_under_recall(df_sweep, min_recall=0.50):
    cand = df_sweep[df_sweep["recall"] >= min_recall].copy()
    if len(cand) == 0:
        return None, cand
    best_row = cand.sort_values(["precision", "recall"], ascending=False).head(1)
    return best_row, cand


In [ ]:

sweep = sweep_threshold_by_top_pct(y_te, score_te)
best_row, cand = pick_best_under_recall(sweep, min_recall=0.50)

best_row


In [59]:
# [None으로 나옴!]
# row = best_row.iloc[0]
# thr = float(row["threshold"])

# y_pred = (np.asarray(score_te) >= thr).astype(int)

# print(f"Selected threshold = {thr:.6f}")
# print(f"top_pct = {float(row['top_pct']):.6f} | precision = {float(row['precision']):.4f} | recall = {float(row['recall']):.4f}\n")
# print(classification_report(y_te, y_pred, digits=4))


AttributeError: 'NoneType' object has no attribute 'iloc'

In [ ]:
# (replace)
def sweep_threshold_by_top_pct(y_true, score, top_pct_grid=None):
    if top_pct_grid is None:
        top_pct_grid = np.array(list(np.linspace(0.0002, 0.10, 120)))  # 0.02% ~ 10%
    y = np.asarray(y_true).astype(int)
    s = np.asarray(score).astype(float)

    rows = []
    for top_pct in top_pct_grid:
        thr = float(np.quantile(s, 1 - top_pct))
        y_pred = (s >= thr).astype(int)

        tp = int(((y_pred == 1) & (y == 1)).sum())
        fp = int(((y_pred == 1) & (y == 0)).sum())
        fn = int(((y_pred == 0) & (y == 1)).sum())

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        rows.append({
            "top_pct": float(top_pct),
            "threshold": thr,
            "precision": float(precision),
            "recall": float(recall),
            "alerts": int(y_pred.sum()),
            "tp": tp, "fp": fp, "fn": fn
        })

    return pd.DataFrame(rows).sort_values("top_pct").reset_index(drop=True)


def pick_operating_point(df_sweep, min_recall=0.50):
    cand = df_sweep[df_sweep["recall"] >= min_recall].copy()
    if len(cand) > 0:
        best = cand.sort_values(["precision", "recall"], ascending=False).head(1)
        return best, "best_precision_under_recall"

    best = df_sweep.sort_values(["recall", "precision"], ascending=False).head(1)
    return best, "max_recall_in_grid"


In [61]:
# (replace)
sweep = sweep_threshold_by_top_pct(y_te, score_te)
best_row, mode = pick_operating_point(sweep, min_recall=0.50)

row = best_row.iloc[0]
thr = float(row["threshold"])
y_pred = (np.asarray(score_te) >= thr).astype(int)

print("mode:", mode)
print(f"Selected threshold = {thr:.6f}")
print(f"top_pct = {float(row['top_pct']):.6f} | precision = {float(row['precision']):.4f} | recall = {float(row['recall']):.4f}\n")
print(classification_report(y_te, y_pred, digits=4))


mode: best_precision_under_recall
Selected threshold = 1.000000
top_pct = 0.058906 | precision = 0.0204 | recall = 0.5290

              precision    recall  f1-score   support

           0     0.9987    0.9330    0.9647    930314
           1     0.0204    0.5290    0.0392      2448

    accuracy                         0.9320    932762
   macro avg     0.5095    0.7310    0.5020    932762
weighted avg     0.9961    0.9320    0.9623    932762



---

### id 추가해서 다시 돌린 후 stage1에서 1로 후보 행 parquet 생성

In [21]:
# id 추가해서 다시 돌린 후 stage1에서 1로 후보 행 parquet 생성

LABEL = "fraud"
ID_COL = "id"

def split_X_y_id(df: pd.DataFrame, label_col=LABEL, id_col=ID_COL):
    df = df.copy()

    if id_col not in df.columns:
        raise KeyError(f"missing {id_col}")

    y = df[label_col].astype("int8").to_numpy()
    ids = df[id_col].to_numpy()

    drop_cols = [label_col, id_col]
    X = df.drop(columns=[c for c in drop_cols if c in df.columns])

    return X, y, ids


In [22]:
train = pd.read_parquet(TRAIN_PATH)
test = pd.read_parquet(TEST_PATH)
X_tr, y_tr, id_tr = split_X_y_id(train)
X_te, y_te, id_te = split_X_y_id(test)


In [24]:
MODEL_DIR = Path("artifacts/stage1_lgb_tuned")

best_model = joblib.load(MODEL_DIR / "lgb_stage1_tuned.joblib")

In [25]:
score_te = best_model.predict(X_te)


[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score

def find_best_threshold_under_recall(y_true, scores, min_recall=0.5):
    thresholds = np.linspace(0, 1, 500)
    best_thr = None
    best_precision = -1

    for thr in thresholds:
        y_pred = (scores >= thr).astype(int)
        recall = recall_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)

        if recall >= min_recall and precision > best_precision:
            best_precision = precision
            best_thr = thr

    return best_thr


In [27]:
thr = find_best_threshold_under_recall(y_te, score_te, min_recall=0.50)
print("Selected threshold:", thr)

Selected threshold: 0.002004008016032064


In [28]:
pass_mask = score_te >= thr

pass_ids = pd.Series(id_te[pass_mask], name="id")

print("n_total:", len(id_te))
print("n_pass:", pass_mask.sum())
print("pass_rate:", pass_mask.mean())

pass_ids.head()

n_total: 932762
n_pass: 9196
pass_rate: 0.009858892193292609


0    17736183
1    17738369
2    17738495
3    17739144
4    18118966
Name: id, dtype: int64

In [29]:
out_df = test.loc[pass_mask, ["id"]].copy()
out_df.to_parquet("artifacts/stage1_pass_ids_test.parquet", index=False)